In [13]:
# Step 0: Install Dependencies (run once if needed)
# %pip install opencv-python moviepy numpy

import cv2
import numpy as np
import os
from moviepy.editor import VideoFileClip, concatenate_videoclips

# Step 1: Configuration Parameters
video_path = r"D:\Ayush\PROJECTS\EE655\Matches\matchclip.mp4"  # Main video
output_dir = "clips"
highlight_output = "highlights.mp4"

clip_duration = 8  # Seconds per replay clip
pre_buffer = 2     # Seconds before replay shown
threshold = 0.5    # Correlation threshold

# Step 2: Load Replay Templates
template_paths = {
    "wicket": r"D:\Ayush\PROJECTS\EE655\templates\replay_wicket.png",
    "four": r"D:\Ayush\PROJECTS\EE655\templates\replay_four.png",
    "six": r"D:\Ayush\PROJECTS\EE655\templates\replay_six.png",
}

templates = {}
template_means = {}
for label, path in template_paths.items():
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (img.shape[1] // 2, img.shape[0] // 2))  # Resize to half
    templates[label] = img.astype(np.float32)
    template_means[label] = np.mean(templates[label])

# Step 3: Create Output Folder
os.makedirs(output_dir, exist_ok=True)

# Step 4: Define Correlation Function
def compute_correlation(frame_gray, template, template_mean):
    frame = cv2.resize(frame_gray, (template.shape[1], template.shape[0]))
    frame = frame.astype(np.float32)
    frame_mean = np.mean(frame)

    numerator = np.sum((frame - frame_mean) * (template - template_mean))
    denominator = np.sqrt(np.sum((frame - frame_mean) ** 2) * np.sum((template - template_mean) ** 2))
    if denominator == 0:
        return 0
    return numerator / denominator

# Step 5: Scan Video for Replay Frames
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_interval = int(fps * 0.5)  # Check every 0.5 seconds

frame_num = 0
timestamps = []  # List of tuples: (timestamp, label)

print("🔍 Scanning video for replay events...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_num % frame_interval == 0:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        for label, template in templates.items():
            corr = compute_correlation(gray, template, template_means[label])
            if corr > threshold:
                timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
                if not timestamps or abs(timestamp - timestamps[-1][0]) > clip_duration:
                    timestamps.append((timestamp, label))
                    print(f"✅ Detected {label.upper()} at {timestamp:.2f}s (corr={corr:.2f})")
                break  # Avoid multiple detections for same frame

    frame_num += 1

cap.release()

# Step 6: Extract Replay Clips
clips = []
video = VideoFileClip(video_path)

print("\n✂️ Extracting clips...")

for i, (ts, label) in enumerate(timestamps):
    start = max(0, ts - pre_buffer)
    end = start + clip_duration

    try:
        subclip = video.subclip(start, end).without_audio()
        if subclip.duration == 0:
            print(f"⚠️ Skipping {label}_{i+1} - zero duration")
            continue

        clip_path = os.path.join(output_dir, f"{label}_{i+1}.mp4")
        subclip.write_videofile(
            clip_path,
            codec="libx264",
            audio=False,
            verbose=False,
            logger=None
        )
        clips.append(VideoFileClip(clip_path))
    except Exception as e:
        print(f"❌ Failed to cut clip at {ts:.2f}s: {e}")

video.close()

# Step 7: Merge All Clips into Final Highlights
if clips:
    print("\n🎬 Generating final highlights...")
    final = concatenate_videoclips(clips)
    final.write_videofile(highlight_output, codec="libx264", audio=False)
    print(f"✅ Highlights saved to: {highlight_output}")
else:
    print("⚠️ No valid replay clips were extracted.")


🔍 Scanning video for replay events...
✅ Detected WICKET at 18.00s (corr=0.62)
✅ Detected WICKET at 334.00s (corr=0.61)
✅ Detected FOUR at 510.00s (corr=0.59)
✅ Detected WICKET at 568.00s (corr=0.73)
✅ Detected WICKET at 652.00s (corr=0.75)
✅ Detected WICKET at 767.00s (corr=0.62)
✅ Detected FOUR at 852.00s (corr=0.51)
✅ Detected FOUR at 883.00s (corr=0.54)
✅ Detected WICKET at 932.00s (corr=0.70)
✅ Detected WICKET at 1071.00s (corr=0.57)



✂️ Extracting clips...
Moviepy - Building video clips\wicket_1.mp4.
Moviepy - Writing video clips\wicket_1.mp4



chunk:  46%|████▌     | 5559/12128 [05:56<00:11, 588.48it/s, now=None]

❌ Failed to cut clip at 18.00s: must be real number, not NoneType


chunk:  46%|████▌     | 5559/12128 [05:56<00:11, 588.48it/s, now=None]

...

Moviepy - Building video clips\four_8.mp4.
Moviepy - Writing video clips\four_8.mp4

❌ Failed to cut clip at 883.00s: must be real number, not NoneType


chunk:  46%|████▌     | 5559/12128 [05:58<00:11, 588.48it/s, now=None]

Moviepy - Building video clips\wicket_9.mp4.
Moviepy - Writing video clips\wicket_9.mp4

❌ Failed to cut clip at 932.00s: must be real number, not NoneType


chunk:  46%|████▌     | 5559/12128 [05:59<00:11, 588.48it/s, now=None]

Moviepy - Building video clips\wicket_10.mp4.
Moviepy - Writing video clips\wicket_10.mp4

❌ Failed to cut clip at 1071.00s: must be real number, not NoneType
⚠️ No valid replay clips were extracted.


In [18]:
# 👉 Complete Highlight Generator without OCR (Template Matching Only)
# Installs (run once):
# pip install opencv-python numpy moviepy tqdm

import cv2
import numpy as np
import os
from moviepy.editor import VideoFileClip, concatenate_videoclips
from tqdm import tqdm

# --- Configuration ---
video_path = r"D:\Ayush\PROJECTS\EE655\Matches\matchclip.mp4"
output_dir = "clips"
frame_dir = "detected_frames"
highlight_output = "highlight_video.mp4"

# Template images for replay detection
template_paths = {
    'wicket': r"D:\Ayush\PROJECTS\EE655\templates\replay_wicket.png",
    'four':   r"D:\Ayush\PROJECTS\EE655\templates\replay_four.png",
    'six':    r"D:\Ayush\PROJECTS\EE655\templates\replay_six.png"
}
# Trophy screen template to ignore
trophy_path = r"D:\Ayush\PROJECTS\EE655\templates\trophy.png"

# Detection parameters
threshold = 0.5           # Correlation threshold for replay templates
trophy_threshold = 0.7    # Correlation threshold for trophy (skip if above)
clip_margin = 5           # Seconds before & after event to include

os.makedirs(output_dir, exist_ok=True)
os.makedirs(frame_dir, exist_ok=True)

# --- Load Templates ---
templates = {}
for label, path in template_paths.items():
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Template not found: {path}")
    # Resize to half for speed
    img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
    templates[label] = img.astype(np.float32)

trophy_img = cv2.imread(trophy_path, cv2.IMREAD_GRAYSCALE)
if trophy_img is None:
    raise FileNotFoundError(f"Trophy template not found: {trophy_path}")
# Resize similarly
h, w = templates[next(iter(templates))].shape
# Crop or resize trophy to match frame sample size
# Here assume trophy full frame: no resize for trophy

def corr_match(frame_gray, tpl):
    # Resize frame to template size
    f = cv2.resize(frame_gray, (tpl.shape[1], tpl.shape[0])).astype(np.float32)
    t = tpl
    num = np.sum((f - f.mean()) * (t - t.mean()))
    den = np.sqrt(np.sum((f - f.mean())**2) * np.sum((t - t.mean())**2))
    return num/den if den!=0 else 0

# --- Scan Video ---
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count / fps

timestamps = []  # (start, end, label)
last_ts = -clip_margin
frame_idx = 0

print("🔍 Scanning for replay events...")
for _ in tqdm(range(frame_count), desc="Scanning frames"):
    ret, frame = cap.read()
    if not ret:
        break
    if frame_idx % int(fps) == 0:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Skip trophy screens
        if corr_match(gray, trophy_img) >= trophy_threshold:
            frame_idx += 1; continue
        # Check each replay type
        best_label, best_corr = None, 0
        for label, tpl in templates.items():
            c = corr_match(gray, tpl)
            if c > best_corr:
                best_corr, best_label = c, label
        if best_corr >= threshold:
            ts = frame_idx / fps
            if ts - last_ts > clip_margin:
                # Save detected frame
                cv2.imwrite(os.path.join(frame_dir, f"{best_label}_{int(ts)}.jpg"), frame)
                print(f"✅ Detected {best_label.upper()} at {ts:.1f}s (corr={best_corr:.2f})")
                timestamps.append((max(0, ts-clip_margin), min(duration, ts+clip_margin), best_label))
                last_ts = ts
    frame_idx += 1
cap.release()

# --- Extract Clips ---
print("\n✂️ Extracting clips...")
clips = []
for idx, (start, end, label) in enumerate(timestamps, 1):
    try:
        clip = VideoFileClip(video_path).subclip(start, end).without_audio()
        clip_path = os.path.join(output_dir, f"{label}_{idx}.mp4")
        print(f"Saving clip {idx}: {label} [{start:.1f}-{end:.1f}s]")
        clip.write_videofile(clip_path, codec="libx264", audio=False, verbose=False, logger=None)
        clips.append(clip)
    except Exception as e:
        print(f"❌ Error extracting clip {idx}: {e}")

# --- Merge Clips ---
if clips:
    print("\n🎬 Merging clips into highlights...")
    final = concatenate_videoclips(clips)
    final.write_videofile(highlight_output, codec="libx264", audio=False)
    print(f"✅ Highlights saved as {highlight_output}")
else:
    print("⚠️ No clips extracted.")


🔍 Scanning for replay events...


✅ Detected FOUR at 18.0s (corr=0.67)


✅ Detected FOUR at 510.0s (corr=0.59)


Scanning frames:  47%|████▋     | 14100/30000 [01:47<02:01, 131.06it/s]


KeyboardInterrupt: 

In [22]:
import cv2
import numpy as np
import os
from moviepy.editor import VideoFileClip, concatenate_videoclips
from tqdm import tqdm

# --- CONFIGURATION ---
video_path = r"D:\Ayush\PROJECTS\EE655\Matches\matchclip.mp4"
output_dir = 'clips'
frame_save_dir = 'detections'
templates = {
    'wicket': cv2.imread(r'D:\Ayush\PROJECTS\EE655\templates\replay_wicket.png', 0),
    'four': cv2.imread(r'D:\Ayush\PROJECTS\EE655\templates\replay_four.png', 0),
    'six': cv2.imread(r'D:\Ayush\PROJECTS\EE655\templates\replay_six.png', 0)
}
trophy_template = cv2.imread(r'D:\Ayush\PROJECTS\EE655\templates\trophy.png', 0)
correlation_threshold = 0.7
trophy_threshold = 0.8
clip_duration = 10
frame_gap_seconds = 0.2  # Check every 0.2s
os.makedirs(output_dir, exist_ok=True)
os.makedirs(frame_save_dir, exist_ok=True)

# --- DETECTION ---
def match_template(frame_gray, template, threshold):
    res = cv2.matchTemplate(frame_gray, template, cv2.TM_CCOEFF_NORMED)
    return np.max(res) > threshold

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise ValueError("❌ Could not open the video. Check the path.")

fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    print("⚠️ FPS is 0, defaulting to 25.")
    fps = 25

frame_gap = int(fps * frame_gap_seconds)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

replay_timestamps = []

print("🔍 Scanning video for replays...")
for i in tqdm(range(0, frame_count, frame_gap)):
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    if not ret:
        continue
    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if match_template(frame_gray, trophy_template, trophy_threshold):
        continue

    for event_type, template in templates.items():
        if match_template(frame_gray, template, correlation_threshold):
            replay_timestamps.append((timestamp, event_type))
            frame_filename = f"{frame_save_dir}/{event_type}_{int(timestamp)}.jpg"
            cv2.imwrite(frame_filename, frame)
            print(f"✅ Detected {event_type.upper()} at {timestamp:.2f}s → saved {frame_filename}")
            break

cap.release()

# --- CLIP EXTRACTION ---
clips = []
video = VideoFileClip(video_path)
print("✂️ Extracting clips...")

for idx, (timestamp, event_type) in enumerate(replay_timestamps):
    start_time = max(timestamp - 2, 0)
    end_time = min(timestamp + clip_duration, video.duration)
    try:
        subclip = video.subclip(start_time, end_time)
        out_path = os.path.join(output_dir, f'{event_type}_{idx+1}.mp4')
        subclip.write_videofile(out_path, codec='libx264', audio=False)
        clips.append(subclip)
    except Exception as e:
        print(f"❌ Failed to cut clip at {timestamp:.2f}s: {e}")

# --- MERGE CLIPS ---
if clips:
    print("🎞️ Merging clips...")
    final_video = concatenate_videoclips(clips)
    final_video.write_videofile("highlights.mp4", codec='libx264', audio=False)
    print("✅ Highlight video generated: highlights.mp4")
else:
    print("⚠️ No valid replay clips were extracted.")


🔍 Scanning video for replays...


✅ Detected FOUR at 652.56s → saved detections/four_652.jpg


✅ Detected FOUR at 652.76s → saved detections/four_652.jpg


✅ Detected FOUR at 652.96s → saved detections/four_652.jpg


✅ Detected FOUR at 653.16s → saved detections/four_653.jpg


...

✅ Detected FOUR at 655.36s → saved detections/four_655.jpg


✅ Detected FOUR at 655.56s → saved detections/four_655.jpg


✅ Detected WICKET at 943.56s → saved detections/wicket_943.jpg


 88%|████████▊ | 5309/6000 [39:21<05:07,  2.25it/s]


KeyboardInterrupt: 

In [25]:
import cv2
import numpy as np
import os
from moviepy.editor import VideoFileClip, concatenate_videoclips
from tqdm import tqdm

# --- CONFIGURATION ---
video_path = r'D:\Ayush\PROJECTS\EE655\Matches\matchclip.mp4'
output_dir = 'clips'
frame_save_dir = 'detections'
templates = {
    'wicket': cv2.imread(r'D:\Ayush\PROJECTS\EE655\templates\replay_wicket.png', 0),
    'four': cv2.imread(r'D:\Ayush\PROJECTS\EE655\templates\replay_four.png', 0),
    'six': cv2.imread(r'D:\Ayush\PROJECTS\EE655\templates\replay_six.png', 0)
}
trophy_template = cv2.imread(r'D:\Ayush\PROJECTS\EE655\templates\trophy.png', 0)
histogram_threshold = 0.7  # Lower value = more strict match
trophy_threshold = 0.8
clip_duration = 10
frame_gap_seconds = 0.2  # Check every 0.2s
os.makedirs(output_dir, exist_ok=True)
os.makedirs(frame_save_dir, exist_ok=True)

# --- HELPER: Histogram-based similarity ---
def hist_similarity(img1, img2):
    img1 = cv2.resize(img1, (200, 100))
    img2 = cv2.resize(img2, (200, 100))
    hist1 = cv2.calcHist([img1], [0], None, [256], [0, 256])
    hist2 = cv2.calcHist([img2], [0], None, [256], [0, 256])
    hist1 = cv2.normalize(hist1, hist1).flatten()
    hist2 = cv2.normalize(hist2, hist2).flatten()
    similarity = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    return similarity

# --- DETECTION ---
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise ValueError("❌ Could not open the video. Check the path.")

fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    print("⚠️ FPS is 0, defaulting to 25.")
    fps = 25

frame_gap = int(fps * frame_gap_seconds)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

replay_timestamps = []

print("🔍 Scanning video for replays...")
for i in tqdm(range(0, frame_count, frame_gap)):
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    if not ret:
        continue
    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if hist_similarity(frame_gray, trophy_template) > trophy_threshold:
        continue

    for event_type, template in templates.items():
        sim = hist_similarity(frame_gray, template)
        if sim > histogram_threshold:
            replay_timestamps.append((timestamp, event_type))
            frame_filename = f"{frame_save_dir}/{event_type}_{int(timestamp)}.jpg"
            cv2.imwrite(frame_filename, frame)
            print(f"✅ Detected {event_type.upper()} at {timestamp:.2f}s → saved {frame_filename}")
            break

cap.release()

# --- CLIP EXTRACTION ---
clips = []
video = VideoFileClip(video_path)
print("✂️ Extracting clips...")

for idx, (timestamp, event_type) in enumerate(replay_timestamps):
    start_time = max(timestamp - 2, 0)
    end_time = min(timestamp + clip_duration, video.duration)
    try:
        subclip = video.subclip(start_time, end_time)
        out_path = os.path.join(output_dir, f'{event_type}_{idx+1}.mp4')
        subclip.write_videofile(out_path, codec='libx264', audio=False)
        clips.append(subclip)
    except Exception as e:
        print(f"❌ Failed to cut clip at {timestamp:.2f}s: {e}")

# --- MERGE CLIPS ---
if clips:
    print("🎞️ Merging clips...")
    final_video = concatenate_videoclips(clips)
    final_video.write_videofile("highlights.mp4", codec='libx264', audio=False)
    print("✅ Highlight video generated: highlights.mp4")
else:
    print("⚠️ No valid replay clips were extracted.")

🔍 Scanning video for replays...


100%|██████████| 6000/6000 [06:55<00:00, 14.43it/s]


✂️ Extracting clips...
⚠️ No valid replay clips were extracted.


In [28]:
import cv2
import numpy as np
import os
from moviepy.editor import VideoFileClip, concatenate_videoclips
from tqdm import tqdm

# ==== Config ====
video_path = r"D:\Ayush\PROJECTS\EE655\Matches\matchclip.mp4"
template_paths = {
    "wicket": r"D:\Ayush\PROJECTS\EE655\templates\replay_wicket.png",
    "four": r"D:\Ayush\PROJECTS\EE655\templates\replay_four.png",
    "six": r"D:\Ayush\PROJECTS\EE655\templates\replay_six.png"
}
trophy_path = r"D:\Ayush\PROJECTS\EE655\templates\trophy.png"

output_dir = "clips"
detected_frames_dir = "detected_frames"
highlight_output = "highlights.mp4"
clip_duration = 8  # seconds
hist_threshold = 0.5
trophy_threshold = 0.7  # Skip if correlation with trophy image is high
frame_gap = 5  # Check every 5th frame (reduce for shorter screen flashes)

# ==== Setup Directories ====
os.makedirs(output_dir, exist_ok=True)
os.makedirs(detected_frames_dir, exist_ok=True)

# ==== Load Templates ====
def load_template(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    return cv2.resize(img, (200, 100))

templates = {name: load_template(p) for name, p in template_paths.items()}
trophy_template = load_template(trophy_path)

# ==== Histogram Correlation ====
def hist_similarity(img1, img2):
    img1 = cv2.resize(img1, (200, 100))
    img2 = cv2.resize(img2, (200, 100))
    hist1 = cv2.calcHist([img1], [0], None, [256], [0, 256])
    hist2 = cv2.calcHist([img2], [0], None, [256], [0, 256])
    cv2.normalize(hist1, hist1)
    cv2.normalize(hist2, hist2)
    return cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)

# ==== Replay Detection ====
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
replay_timestamps = []

print("🔍 Scanning video for replays...")
for i in tqdm(range(0, frame_count, frame_gap), desc="Detecting replays"):
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    if not ret:
        continue
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

    # Skip if trophy template matched
    if hist_similarity(gray, trophy_template) > trophy_threshold:
        continue

    for event_type, template in templates.items():
        score = hist_similarity(gray, template)
        if score > hist_threshold:
            if not replay_timestamps or abs(timestamp - replay_timestamps[-1][1]) > clip_duration:
                replay_timestamps.append((event_type, timestamp))
                frame_path = os.path.join(detected_frames_dir, f"{event_type}_{int(timestamp)}s.jpg")
                cv2.imwrite(frame_path, frame)
            break

cap.release()

# ==== Clip Extraction with Loading Bar ====
print("\n✂️ Extracting clips...")
video_clip = VideoFileClip(video_path)
clips = []

for idx, (event_type, ts) in enumerate(tqdm(replay_timestamps, desc="Extracting clips")):
    try:
        start = max(0, ts - 2)
        end = start + clip_duration
        subclip = video_clip.subclip(start, end)
        clip_path = os.path.join(output_dir, f"{event_type}_{idx+1}.mp4")
        subclip.write_videofile(clip_path, codec="libx264", audio=False, verbose=False, logger=None)
        clips.append(VideoFileClip(clip_path))
    except Exception as e:
        print(f"❌ Failed to cut clip at {ts:.2f}s: {e}")

video_clip.close()

# ==== Merge All Clips ====
if clips:
    final = concatenate_videoclips(clips)
    final.write_videofile(highlight_output, codec="libx264", audio=False)
    print(f"\n🎉 Highlights saved to: {highlight_output}")
else:
    print("⚠️ No valid replay clips were extracted.")


🔍 Scanning video for replays...


Detecting replays: 100%|██████████| 6000/6000 [07:21<00:00, 13.60it/s]



✂️ Extracting clips...


                                                                      
                                                                        
chunk:  46%|████▌     | 5559/12128 [2:11:58<00:11, 588.48it/s, now=None]

Moviepy - Building video clips\four_1.mp4.
MoviePy - Writing audio in four_1TEMP_MPY_wvf_snd.mp3


...

MoviePy - Done.
Moviepy - Writing video clips\four_1.mp4

❌ Failed to cut clip at 431.60s: must be real number, not NoneType


                                                                        
                                                                        
chunk:  46%|████▌     | 5559/12128 [2:12:18<00:11, 588.48it/s, now=None]

Moviepy - Building video clips\four_2.mp4.
MoviePy - Writing audio in four_2TEMP_MPY_wvf_snd.mp3


































































                                                                        
                                                                        
Extracting clips: 100%|██████████| 2/2 [00:26<00:00, 13.11s/it]now=None]


MoviePy - Done.
Moviepy - Writing video clips\four_2.mp4

❌ Failed to cut clip at 943.76s: must be real number, not NoneType
⚠️ No valid replay clips were extracted.
